In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

https://www.kaggle.com/udbhavpangotra/tps-apr21-eda-model


https://www.kaggle.com/hiro5299834/tps-apr-2021-voting-pseudo-labeling

# KAGGLE 스터디 

In [ ]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import lightgbm as lgb
import catboost as ctb
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz

import graphviz
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [ ]:
TARGET = 'Survived'

N_ESTIMATORS = 1000
N_SPLITS = 10
SEED = 2021
EARLY_STOPPING_ROUNDS = 100
VERBOSE = 100

In [ ]:
#랜덤 시드 생성
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

## 데이터 전처리

### lode data

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2021/sample_submission.csv')
#test_df['Survived'] = pd.read_csv("../input/submission-merged3/submission_merged3.csv")['Survived']

all_df = pd.concat([train_df, test_df]).reset_index(drop=True)
#reset_index : 인덱스를 세팅한다. drop=True를 하면 인덱스를 세팅한걸 삭제함. 


In [ ]:
print('Rows and Columns in train dataset:', train_df.shape)
print('Rows and Columns in test dataset:', test_df.shape)

### 결측치 갯수 출력

In [ ]:
print('Missing values per columns in train dataset')
for col in train_df.columns:
    temp_col = train_df[col].isnull().sum()
    print(f'{col}: {temp_col}')
print()
print('Missing values per columns in test dataset')
for col in test_df.columns:
    temp_col = test_df[col].isnull().sum()
    print(f'{col}: {temp_col}')

### Filling missing values

In [ ]:
#나이는 나이의 평균치로 채운다.
all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mean())

#cabin은 문자열을 분할하고, 제일 첫번째 글자를 따와서 넣는다. 결측치엔 X를 넣는다.
#strip() : 양쪽 공백을 지운다. 여기서느 x[0]외엔 다 지우는듯. 
all_df['Cabin'] = all_df['Cabin'].fillna('X').map(lambda x: x[0].strip())


#print(all_df['Ticket'].head(10))
#Ticket, fillna with 'X', split string and take first split 
#split() : 문자열 나누기. 디폴트는 ' '이고, 문자를 가진 데이터들이 전부 띄워쓰기로 구분되어있기때문에 가능. 
all_df['Ticket'] = all_df['Ticket'].fillna('X').map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')

#pclass에 따른 Fare의 평균을 구해서 dictionary형태로 만든다. 
fare_map = all_df[['Fare', 'Pclass']].dropna().groupby('Pclass').median().to_dict()
#fare의 결측치에 본인 행의 pclass 값을 넣고, 그 값을 fare 평균에 맵핑시킨다.  
all_df['Fare'] = all_df['Fare'].fillna(all_df['Pclass'].map(fare_map['Fare']))
#유독 높은 가격이나 낮은 가격이 있기때문에, 이상치의 영향을 줄이기 위해서 Fare에 log를 취해준다.
all_df['Fare'] = np.log1p(all_df['Fare'])


#항구의 결측치를 X로 채운다. 
all_df['Embarked'] = all_df['Embarked'].fillna('X')

#이름은 성만 사용한다.
all_df['Name'] = all_df['Name'].map(lambda x: x.split(',')[0])


In [ ]:
data_1=all_df.loc[all_df['Pclass']==1].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_1)
print()
data_2=all_df.loc[all_df['Pclass']==2].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_2)
print()
data_3=all_df.loc[all_df['Pclass']==3].groupby('Ticket')['Ticket'].count().sort_values(ascending=False)
print(data_3)
print()

## 인코딩 

변수별로 인코딩을 다르게 해준다. 

In [ ]:
label_cols = ['Name', 'Ticket', 'Sex','Pclass','Embarked']
onehot_cols = [ 'Cabin',]
numerical_cols = [ 'Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
#라벨 인코딩 함수. c라는 매개변수를 받아서 맞게 트렌스폼 해준다. 
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

In [ ]:

#StandardScaler(): 평균을 제거하고 데이터를 단위 분산으로 조정한다. 
#그러나 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 되는 함수
scaler = StandardScaler()

onehot_encoded_df = pd.get_dummies(all_df[onehot_cols])
label_encoded_df = all_df[label_cols].apply(label_encoder)
numerical_df = pd.DataFrame(scaler.fit_transform(all_df[numerical_cols]), columns=numerical_cols)
target_df = all_df[TARGET]

all_df = pd.concat([numerical_df, label_encoded_df,onehot_encoded_df, target_df], axis=1)
#all_df = pd.concat([numerical_df, label_encoded_df, target_df], axis=1)

## 모델링

In [ ]:
drop_list=['Survived','Parch']

## not pseudo

In [ ]:
train = all_df.iloc[:100000, :]#0개~100000개
test = all_df.iloc[100000:, :] #100000개~ 
#iloc은 정수형 인덱싱
test = test.drop('Survived', axis=1) #test에서 종속변수를 드랍한다. 
model_results = pd.DataFrame()
folds = 5

In [ ]:
y= train.loc[:,'Survived']
X= train.drop(drop_list,axis=1)

## pseudo

In [ ]:
# y=all_df.loc[:,'Survived']
# X=all_df.drop('Survived',axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.25, random_state=21)


In [ ]:
from sklearn import metrics  
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
params = {
    'metric': 'binary_logloss',
    'n_estimators': N_ESTIMATORS,
    'objective': 'binary',
    'random_state': SEED,
    'learning_rate': 0.01,
    'min_child_samples': 150,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 20,
    'max_depth': 16,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 2,
    'max_bin': 240,
}

In [ ]:
lgbm_model=lgb.LGBMClassifier(**params)
lgbm_model.fit(X_train,y_train)
lgbm_pred=lgbm_model.predict(X_valid)

lgbm_R2=metrics.accuracy_score(y_valid,lgbm_pred)
#lgbm_rmse = np.sqrt(mean_squared_error(lgbm_pred,y_valid))
print('R2 : ',lgbm_R2)
#print("RMSE : ", lgbm_rmse)

In [ ]:
print(len(X_train.columns))
print(X_train.columns)

In [ ]:
def cal_adjust_r2(r2):
    n=80000
    k= len(X_train.columns)
    temp=(1-r2)*(n-1)
    temp2=n-k-1
    ad_r2=1-(temp/temp2)
    return ad_r2

In [ ]:
ad_r2_lgbm=cal_adjust_r2(lgbm_R2)
print(ad_r2_lgbm)

In [ ]:
#NOT Pseudo
train_kf_feature=train.drop(drop_list,axis=1)
train_kf_label=train.loc[:,'Survived']

In [ ]:
#Pseudo
# train_kf_feature=all_df.drop(drop_list,axis=1)
# train_kf_label=all_df.loc[:,'Survived']

In [ ]:
n_iter=0
kfold=StratifiedKFold(n_splits=5)
cv_accuracy=[]
feature_importances = pd.DataFrame()

for train_idx, test_idx in kfold.split(train_kf_feature,train_kf_label):

    X_train=train_kf_feature.iloc[train_idx]
    X_test=train_kf_feature.iloc[test_idx]
    y_train,y_test=train_kf_label.iloc[train_idx],train_kf_label.iloc[test_idx]
    #학습 진행
    lgbm_model.fit(X_train,y_train)
    #예측
    fold_pred=lgbm_model.predict(X_test)
    
    #정확도
    n_iter+=1
    fold_accuracy=metrics.accuracy_score(y_test,fold_pred)
    print("\n {}번째  교차 검증 정확도 : {} , 학습 데이터 크기:{}, 검증 데이터 크기 :{} ".
          format(n_iter,fold_accuracy,X_train.shape[0],X_test.shape[0]))
    cv_accuracy.append(fold_accuracy)
    
    #중요도 
    fi_tmp = pd.DataFrame()
    fi_tmp["feature"] = lgbm_model.feature_name_
    fi_tmp["importance"] = lgbm_model.feature_importances_
    feature_importances = feature_importances.append(fi_tmp)

print('\n 평균 검증 정확도 : ',np.mean(cv_accuracy))
    

In [ ]:
order = list(feature_importances.groupby("feature").
             mean().sort_values("importance", ascending=False).index)
plt.figure(figsize=(10, 10))
sns.barplot(x="importance", y="feature", data=feature_importances, order=order)
plt.title("{} importance".format("LGBMRegressor"))
plt.tight_layout()

### CATBoost


In [ ]:
params_cat = {
    'bootstrap_type': 'Poisson',
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'random_seed': SEED,
    'task_type': 'GPU',
    'max_depth': 8,
    'learning_rate': 0.01,
    'n_estimators': N_ESTIMATORS,
    'max_bin': 280,
    'min_data_in_leaf': 64,
    'l2_leaf_reg': 0.01,
    'subsample': 0.8
}

In [ ]:
#새로운 트레인 valid 셋
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.25, random_state=21)


In [ ]:

cat_model=ctb.CatBoostClassifier(**params_cat)
cat_model.fit(X_train, y_train,verbose=300)
cat_pred=cat_model.predict(X_valid)
print("\n정확도: ", metrics.accuracy_score(y_valid, cat_pred))
cat_R2=metrics.accuracy_score(y_valid,cat_pred)
#lgbm_rmse = np.sqrt(mean_squared_error(lgbm_pred,y_valid))
print('R2 : ',cat_R2)

In [ ]:
cv_accuracy=[]
feature_importances = pd.DataFrame()

for train_idx, test_idx in kfold.split(train_kf_feature,train_kf_label):

    X_train=train_kf_feature.iloc[train_idx]
    X_test=train_kf_feature.iloc[test_idx]
    y_train,y_test=train_kf_label.iloc[train_idx],train_kf_label.iloc[test_idx]
    #학습 진행
    cat_model.fit(X_train,y_train,verbose=500)
    #예측
    fold_pred=cat_model.predict(X_test)
    
    #정확도
    n_iter+=1
    fold_accuracy=metrics.accuracy_score(y_test,fold_pred)
    print("\n {}번째  교차 검증 정확도 : {} , 학습 데이터 크기:{}, 검증 데이터 크기 :{} ".
          format(n_iter,fold_accuracy,X_train.shape[0],X_test.shape[0]))
    cv_accuracy.append(fold_accuracy)
    
    #중요도 . lgbm이랑 명령어가 다르다.
    fi_tmp = pd.DataFrame()
    fi_tmp["feature"] = X_test.columns.to_list()
    fi_tmp["importance"] = cat_model.get_feature_importance()
    feature_importances = feature_importances.append(fi_tmp)

print('\n 평균 검증 정확도 : ',np.mean(cv_accuracy))

In [ ]:
# just to get ideas to improve
order = list(feature_importances.groupby("feature").mean().sort_values("importance", ascending=False).index)
plt.figure(figsize=(10, 10))
sns.barplot(x="importance", y="feature", data=feature_importances, order=order)
plt.title("{} importance".format("CatBoostClassifier"))
plt.tight_layout()

## Submission


In [ ]:
def create_submission(model, test, test_passenger_id, model_name):
    y_pred_test = model.predict_proba(test)[:, 1]
    submission = pd.DataFrame(
        {
            'PassengerId': test_passenger_id, 
            'Survived': (y_pred_test >= 0.5).astype(int),
        }
    )
    submission.to_csv(f"submission_{model_name}.csv", index=False)
    
    return y_pred_test

In [ ]:
test_df.head()

In [ ]:
#X_test=test.drop('Pclass',axis=1)
test = all_df.iloc[100000:, :] #100000개~ 
X_test=test.drop(drop_list,axis=1)
X_test.head()

In [ ]:
test_pred_lightgbm = create_submission(
    lgbm_model, X_test, test_df["PassengerId"], "lightgbm"
)
test_pred_catboost = create_submission(
    cat_model, X_test, test_df["PassengerId"], "catboost"
)

In [ ]:
test_pred_merged = (

    test_pred_lightgbm + 
    test_pred_catboost 
)
test_pred_merged = np.round(test_pred_merged / 2)

In [ ]:
submission = pd.DataFrame(
    {
        'PassengerId': test_df["PassengerId"], 
        'Survived': test_pred_merged.astype(int),
    }
)
submission.to_csv(f"submission_merged3.csv", index=False)